In [3]:
import os
import random
import numpy as np
import torch 
import torch.nn as nn # neural network module
import torch.optim as optim # optimizers
import torch.nn.functional as F 
import torch.autograd as autograd # for stochastci gradient descent 
from torch.autograd import variable
from collections import deque, namedtuple

Architecture of Neural Network

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self, state_size, action_size, seed = 42): # state size will be 8 as per the docs -observation space
        super(NeuralNetwork, self).__init__() # this line is to set inheritance
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, 64) # first later that connects the input layer and the first fully connected layer
        self.fc2 = nn.Linear(64, 64) # connection between the 1st fully connected layer and the 2nd fully connected layer
        self.fc3 = nn.Linear(64, action_size) # connection between the 2nd fully connected layer and the output layer
        # in the above neural network , input is the states, while outputs are the Q values of the actions

    def forward(self, state):
        # The following two lines propagates the signal from the input layer to the 1st fc layer 
        # using the rectification activtion function
        x = self.fc1(state)
        x = F.relu(x)
        x = self.fc2(x) 
        x = F.relu(x) 
        
        return self.fc3(x)


Setting Up the Environment

In [8]:
import gymnasium as gym

env = gym.make('LunarLander-v2')
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
nuber_actions = env.action_space.n

print('State Shape: ', state_shape, ' state size: ',  state_size, 'Number of Actions: ', nuber_actions)

State Shape:  (8,)  state size:  8 Number of Actions:  4


Initializing Hyperparameters of the neural network

In [9]:
learning_rate = 5e-4
minibatch_size = 100 # Number of observations used in one step of the training to update the model parameters
discount_factor = 0.99 # gamma
replay_buffer_size = 1000 # model memory (Experiences)
interpolation_param = 1e-3 # tau

Replay Experience

In [13]:
class ReplayMemody(object):
    def __init__(self, capacity):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.capacity = capacity
        self.memory = []
    
    def push(self, event):
        self.memory.append(event)
        if len(self.memory) > self.capacity:
            del self.memory[0] # when the memory exceeds the buffer, just delete the oldes value basically the 1st ever value in the list

    def sample(self, batch_size):
        experiences = random.sample(self.memory, batch_size)
        states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
        rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
        next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
        dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device) # converted to boolean value before float
        
        return states, actions, rewards, next_states, dones


Implementing DQN class

In [14]:
class Agent():
    def __init__(self, state_size, action_size):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.state_size = state_size
        self.action_size = action_size
        self.local_qnetwork = NeuralNetwork(state_size, action_size).to(self.device)
        self.target_qnetwork = NeuralNetwork(state_size, action_size).to(self.device)
        self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
        self.memory = ReplayMemody(replay_buffer_size)
        self.time_step = 0